## Chargement du dataset et création d'un dataset d'entraînement composé d'un dico avec 2 clés : text et label

In [1]:
import csv
import pandas as pd

file_path_train = "./data/train_submission.csv"
file_path_test = "./data/test_without_labels.csv"

data_train = pd.read_csv(file_path_train)


In [2]:
data_train

,ID,Usage,Text,Label
0,136,Public,Finalment Atena le recibe en l'acropoli d'Ate...,arg
1,62,Public,Jane Laffort fille de Joseph Laffort et d' Ang...,lat
2,74,Public,Сонзэ ялаксонзо - Роджер Джозеф Бошкович - у...,myv
3,40,Public,Mɛniɛ nkùɔ dìì mɔ̀nnì bɛnìtìbɛ̀ kɛ́deè kɛ̀ Nɔ...,tbz
4,30,Public,Ka go dirisa thekniki yeo ya phetogonepiso Le...,tsn
...,...,...,...,...
38849,66,Public,Gorillas es divide in duo species e 4 o 5 subs...,ina
38850,62,Public,Bakebi bamba se nansha nanku etshi mukila wa ...,lua
38851,8,Public,All Äonen de dorvör liggen doot weern fröher...,nds
38852,118,Public,اور اگر یہ لوگ اپنی جانوں پر ظلم کر کے آپکے حض...,urd


## Analyse of the data train

In [3]:
data_train_without_label = data_train[data_train["Label"].isna()]

In [4]:
data_train_without_label

,ID,Usage,Text,Label
1273,26,Public,Āu-lâi in-ūi goân-chū-bîn ê kong-kek 1541 nî ...,NaN
1697,68,Public,Chrzowice sī chi̍t ê tī Pho-lân Kiōng-hô-kok O...,NaN
1976,116,Public,Tī pún só͘-chāi sì-ûi ê tē-hng ū Drávaszerdahe...,NaN
2870,4,Public,Darahanove (Ukraina-gí: ) sī chi̍t ê tī Ukrain...,NaN
3079,46,Public,Chit ūi tī 2010 nî ê jîn-kháu-sò͘ sī 1 113 lâng.,NaN
...,...,...,...,...
36941,6,Public,Arandon sī ūi-tī Hoat-kok Rhône-Alpes toā-khu ...,NaN
37066,126,Public,Haft Cheshmeh-ye Jahanshah (Pho-su-gí: ) sī ch...,NaN
37487,26,Public,Bulair (Bulgaria-gí: ) sī chi̍t ê tī Bulgaria ...,NaN
38375,108,Public,Bô phah-sǹg tī sin-le̍k 10 go̍eh 29 hō ē-po͘ ...,NaN


Il y a 100 instances qui ne sont pas labellisées. 

In [5]:
data_train_without_nan_for_label = data_train.dropna()

In [6]:
number_of_languages = len(data_train["Label"].unique())
print(f"Il y a {number_of_languages} différentes langues dans le dataset de train")

Il y a 390 différentes langues dans le dataset de train


In [7]:
len(data_train["ID"].unique())

80

On observe qu'une langue n'est pas identifiée par son ID car si c'était le cas on aurait le même nombre d'ID différents que de langues différentes. La colonne ID ne semble ainsi nous apporter aucune information

### Analyse stats sur les données labellisées

In [8]:
dataset_sorted_by_number_instances_by_language = data_train_without_nan_for_label.groupby("Label").count().sort_values('Usage', ascending=False)
dataset_sorted_by_number_instances_by_language

,ID,Usage,Text
Label,,,
tgk,300,300,300
guj,200,200,200
tat,200,200,200
crh,200,200,200
kaa,200,200,200
...,...,...,...
gil,2,2,2
toi,1,1,1
kua,1,1,1


On observer que le nombre d'exemples par langue varie énormément. Certaines langues sont sur-représentées (avec 300 instances pour la première) par rapport à d'autres. 

In [9]:
percentage_of_languages_with_at_least_100_instances = len(dataset_sorted_by_number_instances_by_language[dataset_sorted_by_number_instances_by_language["ID"] >= 100])/len(dataset_sorted_by_number_instances_by_language) * 100
print(f"Le pourcentage de langues avec au moins 100 instances est {percentage_of_languages_with_at_least_100_instances}%")

Le pourcentage de langues avec au moins 100 instances est 93.31619537275064%


## Pré-traitement du dataset de train

In [31]:
import string
import re 

def cleaning(text): 
    """
    Fonction pour pré-traiter le texte en enlevant tous les éléments de ponctuation, les chiffres et les double espaces. 
    """

    # Liste de ponctuation à inclure pour les langues asiatiques
    asian_punctuation = "，。？！《》【】（）；：、。"
    text_without_dash = text.replace('-', ' ')
    text_without_punctuation = text_without_dash.translate(str.maketrans('', '', string.punctuation + asian_punctuation))
    text_without_punctuation_figures = text_without_punctuation.translate(str.maketrans('','', string.digits))
    
    # # Supprimer les emojis
    # emoji_pattern = re.compile(
    #     "["
    #     u"\U0001F600-\U0001F64F"  # emoticons
    #     u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    #     u"\U0001F680-\U0001F6FF"  # transport & map symbols
    #     u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    #     u"\U00002702-\U000027B0"  # dingbats
    #     u"\U000024C2-\U0001F251"
    #     "]+", flags=re.UNICODE
    # )

    # text_without_emojis = emoji_pattern.sub(r'', text_without_punctuation_figures)

    text_cleaned = re.sub(r'\s+', ' ', text_without_punctuation_figures).strip()
    text_cleaned = text_cleaned.lower()

    return(text_cleaned)


### Création d'un ensemble de mots anglais pour pouvoir enlever les mots anglais dans les phrases avec des mots anglais mélangés à d'autres langues

In [11]:
import nltk
from nltk.corpus import words

# Télécharger la liste des mots en anglais (une seule fois nécessaire)
nltk.download('words')

# Liste des mots en anglais
english_words = set(word.lower() for word in words.words())


[nltk_data] Downloading package words to
[nltk_data]     /Users/hippolytelecomte/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
data_ang = data_train_without_nan_for_label[data_train_without_nan_for_label["Label"] == 'eng']["Text"]

# Collecte des mots uniques
for text in data_ang:
    for word in text.split():
        english_words.add(word.lower())



In [13]:
def remove_most_english_words(text): 
    """
    Fonction pour enlever les mots anglais lorsque la langue du texte n'est pas l'anglais. 
    """
    tokens = text.split() 
    filtered_tokens = [word for word in tokens if word.lower() not in english_words]

    return ' '.join(filtered_tokens)

## Première approche avec CountVectorizer et MultinomialNB

Séparation entre le train et le val

In [14]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(data_train_without_nan_for_label, test_size=0.2, random_state=42)

Application du pré-traitement sur tout le dataframe

In [32]:
from tqdm import tqdm
tqdm.pandas()  

def pre_processing(df, remove_espace = True): 
    df['Text'] = df['Text'].apply(cleaning)
    
    df['Text'] = df.progress_apply(
        lambda row: remove_most_english_words(row['Text']) if row['Label'] != 'eng' else row['Text'], axis=1
    )
    
    if remove_espace: 
        df['Text'] = df['Text'].str.replace(' ', '', regex=False)
    
    return df


In [16]:
train_set_first_version = train_set.copy()
val_set_first_version = val_set.copy()
train_set_first_version = pre_processing(train_set_first_version, remove_espace=True)
val_set_first_version = pre_processing(val_set_first_version, remove_espace=True)

100%|██████████| 7751/7751 [00:00<00:00, 84281.01it/s]


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer(analyzer="char", ngram_range=(2, 6), max_features=10000)
x_train = train_set_first_version['Text'].tolist()
y_train = train_set_first_version['Label'].tolist()
x_val = val_set_first_version['Text'].tolist()
y_val = val_set_first_version['Label'].tolist()
y_total = y_train + y_val

# converting categorical variables to numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_total)

y_train = le.transform(y_train)
y_val = le.transform(y_val)
label_mapping = dict(zip(le.classes_, range(len(le.classes_))))


x_train = vectorizer.fit_transform(x_train)
x_val = vectorizer.transform(x_val)

In [18]:
import numpy as np

# Obtenir les indices des classes présentes dans y_val
present_classes = np.unique(y_val)

# Extraire uniquement les noms correspondants
filtered_target_names = [le.classes_[i] for i in present_classes]

In [19]:
naive_bayes = MultinomialNB(alpha= 0.0001, fit_prior = False) 
naive_bayes.fit(x_train,y_train)

MultinomialNB(alpha=0.0001, fit_prior=False)

In [20]:
dataset_sorted_by_number_instances_by_language.loc["yue"]

ID       100
Usage    100
Text     100
Name: yue, dtype: int64

In [21]:
from sklearn.metrics import accuracy_score, classification_report

predictions = naive_bayes.predict(x_val)
accuracy = accuracy_score(y_val, predictions)
print("Accuracy:", accuracy)

# Générer le rapport de classification sous forme de dictionnaire
report = classification_report(y_val, predictions, target_names=filtered_target_names, output_dict=True)

# Filtrer les classes (en excluant 'accuracy', 'macro avg', 'weighted avg')
filtered_report = {label: metrics for label, metrics in report.items() if isinstance(metrics, dict)}

# Trier les langues par F1-score de manière décroissante
sorted_report = sorted(filtered_report.items(), key=lambda x: x[1]['f1-score'], reverse=True)

# Afficher le rapport trié
print("Classification Report (trié par F1-score décroissant):\n")
for label, metrics in sorted_report:
    print(f"{label}: F1-score = {metrics['f1-score']:.4f}, Precision = {metrics['precision']:.4f}, Recall = {metrics['recall']:.4f}, Support = {metrics['support']}")


Accuracy: 0.6739775512837053
Classification Report (trié par F1-score décroissant):

ahk: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
arn: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 18.0
cab: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 28.0
cak: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
csy: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 31.0
ctu: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 24.0
cuk: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
div: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 26.0
djk: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 14.0
guc: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 24.0
gym: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 10.0
hnj: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 2.0
hui:

/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [22]:
val_set[val_set['Label'] == "lzh"]

,ID,Usage,Text,Label
25031,32,Public,一九九一年蘇聯解體，俄羅斯聯邦改立米哈伊爾·格林卡之《愛國歌》為國歌。及公元二千年，俄國杜馬...,lzh
29464,10,Public,黃曉明，二〇〇八年,lzh
32061,92,Public,一九七九年九月，入政治學院學之，屬基本系，一九八零年八月卒業也。一九八五年六月兵駐南京。一九...,lzh
15675,138,Public,東島之飼貓也，其有識而最古者，宇多天皇。,lzh
36083,56,Public,二月廿二，山西古交礦難，亡者七十四，傷一百十四。,lzh
29728,16,Public,《舊五代史·卷一百二十六·周書列傳六》,lzh
22963,12,Public,原創動力之油管，英文,lzh
993,54,Public,召還，復拜右僕射。因入謝，宣仁后簾中諭曰：「或謂卿必先引用王覿、彭汝礪，卿宜與呂大防一心。」...,lzh
23640,80,Public,渡島總合振興局,lzh
16143,6,Public,三子：啟昭，康熙五十五年襲奉恩將軍，乾隆二十六年卒,lzh


## Deuxième approche avec SentencePiece comme tokenizer

### Génération d'un fichier brut .txt pour entraîner SentencePiece

In [23]:
# Extraire uniquement la colonne "Text"
corpus_path = "corpus_multilingue.txt"  # Chemin de sortie pour le corpus
data_train_preprocessed_for_corpus = data_train.copy()
data_train_preprocessed_for_corpus = pre_processing(data_train_preprocessed_for_corpus, remove_espace=False)
data_train_preprocessed_for_corpus["Text"].dropna().to_csv(corpus_path, index=False, header=False, sep="\n")

print(f"Corpus enregistré : {corpus_path}, avec {len(data_train)} phrases.")

100%|██████████| 38854/38854 [00:00<00:00, 84850.72it/s]


Corpus enregistré : corpus_multilingue.txt, avec 38854 phrases.


### Entraînement de SentencePiece et chargement du modèle

In [25]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input='./data/corpus_multilingue.txt',  
    model_prefix='sp_model',
    vocab_size=30000,  
    character_coverage=1.0,  
    model_type='unigram'  
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/corpus_multilingue.txt
  input_format: 
  model_prefix: sp_model
  model_type: UNIGRAM
  vocab_size: 30000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_priva

In [26]:
sp = spm.SentencePieceProcessor(model_file='sp_model.model')

def sentencepiece_tokenize(text):
    """Tokenise un texte en sous-mots avec SentencePiece"""
    return ' '.join(sp.encode(text, out_type=str))

In [33]:
train_set_second_version = train_set.copy()
val_set_second_version = val_set.copy()
train_set_second_version = pre_processing(train_set_second_version, remove_espace=False)
val_set_second_version = pre_processing(val_set_second_version, remove_espace=False)

100%|██████████| 7751/7751 [00:00<00:00, 79550.28it/s]


In [34]:
# Appliquer SentencePiece à ton dataset
train_set_second_version['Text'] = train_set_second_version['Text'].progress_apply(sentencepiece_tokenize)
val_set_second_version['Text'] = val_set_second_version['Text'].progress_apply(sentencepiece_tokenize)


100%|██████████| 7751/7751 [00:00<00:00, 26871.39it/s]


In [35]:
vectorizer_sp = CountVectorizer(analyzer="char", ngram_range=(1, 3), max_features=10000)
x_train_sp = train_set_second_version['Text'].tolist()
y_train_sp = train_set_second_version['Label'].tolist()
x_val_sp = val_set_second_version['Text'].tolist()
y_val_sp = val_set_second_version['Label'].tolist()
y_total_sp = y_train_sp + y_val_sp

# converting categorical variables to numerical
from sklearn.preprocessing import LabelEncoder
le_sp = LabelEncoder()
le_sp.fit(y_total_sp)

y_train_sp = le_sp.transform(y_train_sp)
y_val_sp = le_sp.transform(y_val_sp)
label_mapping = dict(zip(le_sp.classes_, range(len(le_sp.classes_))))


x_train_sp_vectorized = vectorizer_sp.fit_transform(x_train_sp)
x_val_sp_vectorized = vectorizer_sp.transform(x_val_sp)

naive_bayes_sp = MultinomialNB(alpha= 0.0001, fit_prior = False) 
naive_bayes_sp.fit(x_train_sp_vectorized,y_train_sp)


MultinomialNB(alpha=0.0001, fit_prior=False)

In [36]:
from sklearn.metrics import accuracy_score, classification_report

predictions_sp = naive_bayes_sp.predict(x_val_sp_vectorized)
accuracy_sp = accuracy_score(y_val_sp, predictions_sp)
print("Accuracy:", accuracy_sp)

Accuracy: 0.7268739517481615


In [37]:
present_classes_sp = np.unique(np.concatenate((y_val_sp, predictions_sp)))

# Extraire uniquement les noms correspondants
filtered_target_names_sp = [le_sp.classes_[i] for i in present_classes_sp]

In [38]:
# Générer le rapport de classification sous forme de dictionnaire
report_sp = classification_report(y_val_sp, predictions_sp, target_names=filtered_target_names_sp, output_dict=True)

# Filtrer les classes (en excluant 'accuracy', 'macro avg', 'weighted avg')
filtered_report = {label: metrics for label, metrics in report_sp.items() if isinstance(metrics, dict)}

# Trier les langues par F1-score de manière décroissante
sorted_report = sorted(filtered_report.items(), key=lambda x: x[1]['f1-score'], reverse=True)

# Afficher le rapport trié
print("Classification Report (trié par F1-score décroissant):\n")
for label, metrics in sorted_report:
    print(f"{label}: F1-score = {metrics['f1-score']:.4f}, Precision = {metrics['precision']:.4f}, Recall = {metrics['recall']:.4f}, Support = {metrics['support']}")


Classification Report (trié par F1-score décroissant):

ahk: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
alt: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 21.0
aoj: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 23.0
arn: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 18.0
asm: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 15.0
bpy: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
bzj: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 13.0
cab: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 28.0
cak: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
ctu: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 24.0
cuk: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 20.0
div: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 26.0
ewe: F1-score = 1.0000, Precisio

/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sample

In [105]:
val_set_second_version[val_set_second_version['Label'] == 'cos']

,ID,Usage,Text,Label
3785,40,Public,▁sha r p r ight wi tho ut ex it ▁vu tari ▁tutt...,cos
12740,12,Public,▁il ▁mar s char ▁ vid ▁– ▁fest as ▁music a ▁è ...,cos
621,132,Public,▁mis ure ▁sco ppi u ▁metro ▁i stan bul,cos
9616,16,Public,▁ghea z ã ▁ ĭ á ▁zi ▁s f ▁– ▁ve dz ▁e a zi,cos
31460,90,Public,▁km ▁centro ▁ame tz aga ▁zu ia,cos
20114,154,Public,▁in el ▁ar gin t ▁aur ▁ame ti st,cos
22421,2,Public,▁li sta ta ▁o ri ▁ jurca ▁ra lu ca ▁of er te ▁...,cos
36321,112,Public,▁har hu es ▁te uf ert ▁g mb h ▁ro ti ▁ro ti te...,cos
18579,158,Public,▁ru t ko w ski,cos
16301,82,Public,▁ ris pun che mu ▁ut ili z à ▁col tu ▁ rin fu ...,cos


Test avec SVC

In [ ]:
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer_sp_2 = TfidfVectorizer(analyzer="char", ngram_range=(1,3), max_features=10000)
# svc_model = SVC(kernel='linear')

# x_train_svc_vectorized = vectorizer_sp_2.fit_transform(x_train_sp)
# x_val_svc_vectorized = vectorizer_sp_2.transform(x_val_sp)

# svc_model.fit(x_train_svc_vectorized,y_train_sp)

# predictions_svc = svc_model.predict(x_val_svc_vectorized)
# accuracy_svc = accuracy_score(y_val_sp, predictions_svc)
# print("Accuracy:", accuracy_svc)


Test avec SGDClassifier

In [ ]:
# from sklearn.linear_model import SGDClassifier

# vectorizer_sp_2 = TfidfVectorizer(analyzer="char", ngram_range=(1,3), max_features=10000)
# sgdclassifier_model = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=5, tol=None)

# x_train_sgdc_vectorized = vectorizer_sp_2.fit_transform(x_train_sp)
# x_val_sgdc_vectorized = vectorizer_sp_2.transform(x_val_sp)

# # Entraînement avec suivi de progression
# epochs = 5  # Nombre d'époques
# for epoch in range(epochs):
#     print(f"Epoch {epoch + 1}/{epochs}")
#     sgdclassifier_model.fit(x_train_sgdc_vectorized, y_train_sp)
#     val_accuracy = accuracy_score(y_val_sp, sgdclassifier_model.predict(x_val_sgdc_vectorized))
#     print(f"Val Accuracy: {val_accuracy:.4f}\n")




Epoch 1/5
Val Accuracy: 0.6957

Epoch 2/5
Val Accuracy: 0.6898

Epoch 3/5
Val Accuracy: 0.6929

Epoch 4/5
Val Accuracy: 0.6966

Epoch 5/5
Val Accuracy: 0.6941

